In [2]:
#dependencies
import pandas as pd
import json
from datetime import datetime
import re

In [3]:
#csvs from sql queries
kaddex = "swaps_kaddex.csv"
kdx = pd.read_csv(kaddex)

kdswap = "swaps_kdswap.csv"
kds = pd.read_csv(kdswap)

brodex = "swaps_bro-dex.csv"
bro = pd.read_csv(brodex)

In [4]:
def process(csv):
    df = pd.read_csv(csv)

    def process_tokens(row):
        try:
            if pd.notnull(row['result']):
                parsed = json.loads(row['result'])
                if isinstance(parsed.get("data"), list) and len(parsed["data"]) == 2:
                    t0, t1 = parsed["data"][0], parsed["data"][1]
                    amt0 = t0.get("amount", 0)
                    amt1 = t1.get("amount", 0)
                    if isinstance(amt0, dict): amt0 = float(amt0.get("decimal", 0))
                    if isinstance(amt1, dict): amt1 = float(amt1.get("decimal", 0))
                    return pd.Series([t0.get("token", ""), amt0, t1.get("token", ""), amt1])
        except Exception:
            pass

        try:
            if isinstance(row['code'], str) and 'swap-exact' in row['code']:
                numbers = re.findall(r'[-+]?\d*\.\d+|\d+', row['code'])
                token_matches = re.findall(r'\[(.*?)\]', row['code'])

                if len(numbers) >= 2 and token_matches:
                    token_list = token_matches[0].split()
                    if len(token_list) >= 2:
                        token0 = token_list[0].strip('",()')
                        token1 = token_list[1].strip('",()')
                        token0amount = float(numbers[0])
                        token1amount = float(numbers[1])
                        return pd.Series([token0, token0amount, token1, token1amount])
        except Exception:
            pass

        return pd.Series(["", 0, "", 0])

    df_tokens = df.apply(process_tokens, axis=1)
    df_tokens.columns = ['token0', 'token0amount', 'token1', 'token1amount']

    df['time'] = pd.to_datetime(df['creationtime'], unit='s', errors='coerce')
    df['requestkey'] = df['requestkey'].apply(
        lambda rk: f"https://explorer.kadena.io/mainnet/transaction/{rk}"
    )

    final_df = pd.concat([
        df[['sender', 'requestkey']],
        df_tokens,
        df['time']
    ], axis=1)

    cleaned_df = final_df[
        (final_df['token0'] != "") &
        (final_df['token1'] != "") &
        (final_df['token0amount'] != 0) &
        (final_df['token1amount'] != 0)
    ].copy()

    return cleaned_df

In [5]:
def process_brodex(csv):
    df = pd.read_csv(csv)

    successful = df[df['result'].str.contains('"status": "success"')].copy()

    def process_tokens_brodex(row):
        try:
            code = row['code']
            if 'bro-dex-wrapper' not in code or 'cancel-order' in code:
                return pd.Series(["", 0, "", 0])

            match = re.search(r'bro-dex-wrapper-([A-Z0-9]+)-([A-Z0-9]+)-M\.(buy|sell)', code)
            if not match:
                return pd.Series(["", 0, "", 0])
            tokenA, tokenB, side = match.groups()

            nums = re.findall(r'[-+]?\d*\.\d+|\d+', code)
            if len(nums) < 2:
                return pd.Series(["", 0, "", 0])
            price = float(nums[-2])
            amount = float(nums[-1])

            if side == 'buy':
                return pd.Series([tokenB, amount, tokenA, price])
            else:
                return pd.Series([tokenA, price, tokenB, amount])
        except Exception:
            return pd.Series(["", 0, "", 0])

    token_df = successful.apply(process_tokens_brodex, axis=1)
    token_df.columns = ['token0', 'token0amount', 'token1', 'token1amount']

    successful['time'] = pd.to_datetime(successful['creationtime'], unit='s', errors='coerce')
    successful['requestkey'] = successful['requestkey'].apply(
        lambda rk: f"https://explorer.kadena.io/mainnet/transaction/{rk}"
    )

    final_df = pd.concat([
        successful[['sender', 'requestkey']],
        token_df,
        successful['time']
    ], axis=1)

    cleaned_df = final_df[
        (final_df['token0'] != "") &
        (final_df['token1'] != "") &
        (final_df['token0amount'] > 0) &
        (final_df['token1amount'] > 0)
    ].copy()

    return cleaned_df

Mercatus

In [6]:
kaddex_df = process("swaps_kaddex.csv")
kaddex_df.to_csv("kdx_swaps.csv", index=False)

kaddex_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
0,k:b46e0140cf675d78c3b88953a5942c737e1f330ff700...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,4639.0,coin,3.463681e+01,2025-06-13 02:31:49
1,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.397144e+00,2025-06-13 02:32:38
2,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.372858e+00,2025-06-13 02:32:47
3,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.348692e+00,2025-06-13 02:32:56
4,k:fea29be0591af42646ca25dce0e356e77eae3ef8aff7...,https://explorer.kadena.io/mainnet/transaction...,coin,163.0,n_625e9938ae84bdb7d190f14fc283c7a6dfc15d58.ktoshi,1.375245e+09,2025-06-13 03:58:01


KDSwap

In [7]:
kdswap_df = process("swaps_kdswap.csv")
kdswap_df.to_csv("kds_swaps.csv", index=False)

kdswap_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
0,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,n_582fed11af00dc626812cd7890bb88e72067f28c.bro,0.0149,coin,32.068918,2025-06-13 19:43:31
1,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,coin,10.0000,n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI,4339.061430,2025-06-13 14:24:54
2,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI,4415.4181,coin,10.114271,2025-06-13 14:31:50
3,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,coin,20.0000,kdlaunch.kdswap-token,9231.846460,2025-06-13 14:42:06
4,k:b46e0140cf675d78c3b88953a5942c737e1f330ff700...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,3209.0000,coin,23.924557,2025-06-13 02:31:49


Bro-Dex

In [8]:
brodex_df = process_brodex("swaps_bro-dex.csv")
brodex_df.to_csv("bro_swaps.csv",index=False)

brodex_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
15,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0100,KDA,2.260949e+03,2025-06-13 04:16:30
16,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0200,HERON,9.221481e+06,2025-06-13 04:16:33
19,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0177,HERON,9.087505e+06,2025-06-13 04:16:33
20,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,KDA,2194.1212,BRO,9.110000e-03,2025-06-13 04:16:31
21,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0100,KDA,2.294282e+03,2025-06-13 04:16:30


Swaps over X KDA

In [9]:
swap_threshold = 10 #define X

kdx_token0_high = kaddex_df[(kaddex_df['token0'] == 'coin') & (kaddex_df['token0amount'] > swap_threshold)]
kdx_token1_high = kaddex_df[(kaddex_df['token1'] == 'coin') & (kaddex_df['token1amount'] > swap_threshold)]

kdx_accounts_token0 = set(kdx_token0_high['sender'].unique())
kdx_accounts_token1 = set(kdx_token1_high['sender'].unique())
kdx_all_accounts = kdx_accounts_token0 | kdx_accounts_token1

kds_token0_high = kdswap_df[(kdswap_df['token0'] == 'coin') & (kdswap_df['token0amount'] > swap_threshold)]
kds_token1_high = kdswap_df[(kdswap_df['token1'] == 'coin') & (kdswap_df['token1amount'] > swap_threshold)]

kds_accounts_token0 = set(kds_token0_high['sender'].unique())
kds_accounts_token1 = set(kds_token1_high['sender'].unique())
kds_all_accounts = kds_accounts_token0 | kds_accounts_token1

bro_token0_high = brodex_df[(brodex_df['token0'] == 'KDA') & (brodex_df['token0amount'] > swap_threshold)]
bro_token1_high = brodex_df[(brodex_df['token1'] == 'KDA') & (brodex_df['token1amount'] > swap_threshold)]

bro_accounts_token0 = set(bro_token0_high['sender'].unique())
bro_accounts_token1 = set(bro_token1_high['sender'].unique())
bro_all_accounts = bro_accounts_token0 | bro_accounts_token1

swap_accounts = kdx_all_accounts | kds_all_accounts | bro_all_accounts

In [13]:
display(swap_accounts)

{'k:0349744a50c7e1705957b7ed3502fe909b1d8f257c2aef7cffb8f80e6a1ef9cf',
 'k:03df480e0b300c52901fdff265f0460913fea495f39972321698740536cc38e3',
 'k:07d6a3ebaeae0232a8ec4217d582803b72ee96a61d53dba8e717938da4928a9f',
 'k:0d5017ff4cab38efd860cf9b7431778daf2548c0a6460bd655c8ef1ae6119e53',
 'k:13ebe6a6a5ebde1ab649c619080089870ebd19edfb91fe50a65b7e0ddd20e32d',
 'k:2652d1604d6986f9390163b03684074f185f02016b1be54264cc20c0d06861cf',
 'k:279176bc553dc28ec5948f3f21ab4bb6c3367a5b5d23f8f91656f8216b83890a',
 'k:3bba6b8535a55f0b704e9dfea44f675d1101f447952ee5fd950e423663488d3d',
 'k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f32eb9f6da6114729231c1',
 'k:481567361eaa2988c4043aa27b2a0d07c944c0f3ee739deadb81a7b05e451e20',
 'k:49d21023725ad5dc6ce1170cb2f3a40c49d3c68eea70a9777f3ea27061a0fcb6',
 'k:51a1a6c692e7c14a083eff18bb37278ccd75b3be80fb571b7e43834af947c13b',
 'k:540db596e796729ebca757b49b12d1e0dca69d41d2dc73e648eea12ca9ec4f1c',
 'k:5d96a61871e582fb285a5b7fda79f1925bc0f68745f7a3a0ebea71d094f2718b',
 'k:5e

Users with over X KDA in volume

In [20]:
volume_threshold = 10 #define X

kdx_token0_kda = kaddex_df[kaddex_df['token0'] == 'coin'][['sender', 'token0amount']].copy()
kdx_token0_kda.rename(columns={'token0amount': 'kda_amount'}, inplace=True)

kdx_token1_kda = kaddex_df[kaddex_df['token1'] == 'coin'][['sender', 'token1amount']].copy()
kdx_token1_kda.rename(columns={'token1amount': 'kda_amount'}, inplace=True)

kdx_all_kda = pd.concat([kdx_token0_kda, kdx_token1_kda], ignore_index=True)

kdx_volume_by_account = kdx_all_kda.groupby('sender')['kda_amount'].sum().reset_index()
kdx_high_volume = kdx_volume_by_account[kdx_volume_by_account['kda_amount'] > volume_threshold]

kds_token0_kda = kdswap_df[kdswap_df['token0'] == 'coin'][['sender', 'token0amount']].copy()
kds_token0_kda.rename(columns={'token0amount': 'kda_amount'}, inplace=True)

kds_token1_kda = kdswap_df[kdswap_df['token1'] == 'coin'][['sender', 'token1amount']].copy()
kds_token1_kda.rename(columns={'token1amount': 'kda_amount'}, inplace=True)

kds_all_kda = pd.concat([kds_token0_kda, kds_token1_kda], ignore_index=True)

kds_volume_by_account = kds_all_kda.groupby('sender')['kda_amount'].sum().reset_index()
kds_high_volume = kds_volume_by_account[kds_volume_by_account['kda_amount'] > volume_threshold]

bro_token0_kda = brodex_df[brodex_df['token0'] == 'KDA'][['sender', 'token0amount']].copy()
bro_token0_kda.rename(columns={'token0amount': 'kda_amount'}, inplace=True)

bro_token1_kda = brodex_df[brodex_df['token1'] == 'KDA'][['sender', 'token1amount']].copy()
bro_token1_kda.rename(columns={'token1amount': 'kda_amount'}, inplace=True)

bro_all_kda = pd.concat([bro_token0_kda, bro_token1_kda], ignore_index=True)

bro_volume_by_account = bro_all_kda.groupby('sender')['kda_amount'].sum().reset_index()
bro_high_volume = bro_volume_by_account[bro_volume_by_account['kda_amount'] > volume_threshold]

all_kda_transactions = pd.concat([kdx_all_kda, kds_all_kda, bro_all_kda], ignore_index=True)

total_volume_by_account = all_kda_transactions.groupby('sender')['kda_amount'].sum().reset_index()
total_volume_by_account = total_volume_by_account.sort_values('kda_amount', ascending=False)

high_volume_accounts = total_volume_by_account[total_volume_by_account['kda_amount'] > volume_threshold]

In [21]:
display(high_volume_accounts)

,sender,kda_amount
19,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,423644.836500
23,k:93face383ca119d68ab1cb21de1a711bd00b8e4bd47e...,53577.069763
41,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,23443.904836
26,k:a57e1b88758865333f88630026c9f59f87c5a924764e...,14100.000000
4,k:13ebe6a6a5ebde1ab649c619080089870ebd19edfb91...,6746.900000
31,k:b46e0140cf675d78c3b88953a5942c737e1f330ff700...,6165.076842
16,k:5e62f1da2ef3b35bdfbfead6f138c7dacbce8a1fefa1...,5094.216850
8,k:3bba6b8535a55f0b704e9dfea44f675d1101f447952e...,4937.000000
15,k:5d96a61871e582fb285a5b7fda79f1925bc0f68745f7...,3514.487470
10,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,2482.666960
